### Ground Truth Coverage

In [ ]:
import pandas as pd
df = pd.read_csv("df.csv")

Ground Truth: Lim_and_OR_ground_truth_final ||
LLM Generated Limitations: master_agent_ext_analy_rev_cit

In [ ]:
# making lists of list 'master_agent' text
import re

# make sure the output column exists
df['master_agent_ext_analy_rev_cit_list'] = None

for row_idx in range(len(df)):
    raw = df.at[row_idx, "master_agent_ext_analy_rev_cit"]
    # skip if missing or not a string
    if not isinstance(raw, str):
        df.at[row_idx, 'master_agent_ext_analy_rev_cit_list'] = []
        continue

    # split on double-newline before a numbered item
    parts = re.split(r'\n\n(?=\d+\.)', raw.strip())

    lim_list = []
    for part in parts:
        m = re.match(r'(\d+)\.\s*(.*)', part, re.S)
        if not m:
            continue
        num  = int(m.group(1))
        text = m.group(2).strip()
        lim_list.append([num, text])

    df.at[row_idx, 'master_agent_ext_analy_rev_cit_list'] = lim_list


In [ ]:
# making lists of list 'ground truth' text

import re

# ensure the output column exists
df['Lim_and_OR_ground_truth_list'] = None

for row_idx in range(len(df)):
    raw = df.at[row_idx, "Lim_and_OR_ground_truth_final"]
    # skip non-strings
    if not isinstance(raw, str):
        df.at[row_idx, 'Lim_and_OR_ground_truth_list'] = []
        continue

    # split on double-newline before a numbered item
    parts = re.split(r'\n\n(?=\d+\.)', raw.strip())

    lim_list = []
    for part in parts:
        m = re.match(r'(\d+)\.\s*(.*)', part, flags=re.S)
        if not m:
            continue
        num  = int(m.group(1))
        text = m.group(2).strip()
        lim_list.append([num, text])

    df.at[row_idx, 'Lim_and_OR_ground_truth_list'] = lim_list


In [ ]:
# making combinations from 'ground truth' and llm generated text'

df['combined'] = [[] for _ in range(len(df))]

# Generate combinations for each row
for i in range(len(df)):
    combined_list = []
    list1 = df["Lim_and_OR_ground_truth_list"][i]
    list2 = df["master_agent_ext_analy_rev_cit_list"][i]

    # Generate all possible combinations
    for item1 in list1:
        for item2 in list2:
            combined_list.append((item1, item2))

    # Store the first 100 combinations (or all if fewer)
    df.at[i, 'combined'] = combined_list  # Truncate if needed

In [ ]:
import os
import base64
import time
import pandas as pd
# from openai import AzureOpenAI, RateLimitError

import os
import time
from openai import OpenAI

# Set up OpenAI API
os.environ['OPENAI_API_KEY'] = ''
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# Define the OpenAI streaming function for GPT-4o-mini
def run_critic_openai(prompt: str):
    summary_text = ""
    stream = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt}
        ],
        stream=True,
        temperature=0
    )
    for chunk in stream:
        summary_text += chunk.choices[0].delta.content or ""
    return summary_text.strip()

# Now your batch‐processing loop:
all_generated_summary = []
start_time = time.time()

import json

llm_results = []
df['LLM_eval_master_agent_ext_analy_rev_cit_list'] = ''
for idx in range(len(df)): # len(df)
    print("idx is",idx)
    pairs = df.at[idx, 'combined']   # assume this is List[Tuple[list, list]]
    if not isinstance(pairs, list) or not pairs:
        llm_results.append(None)
        continue

    # build the named-pairs block in one go
    formatted = "\n".join(
        f"Pair {i+1}:\n  List1: {first}\n  List2: {second}"
        for i, (first, second) in enumerate(pairs)
    )

    prompt = (
        "For each of the following pairs, answer “Yes” if List1 contains a topic or limitation\n"
        "from List2, or List2 contains a topic or limitation from from List1; otherwise answer “No”.\n"
        "Respond *only* with a JSON object mapping each Pair name to “Yes” or “No”.\n\n"
        "Pairs:\n"
        f"{formatted}"
    )

    # single call per row
    resp_text = run_critic_openai(prompt)
    llm_results.append(resp_text)

    df.at[idx, 'LLM_eval_master_agent_ext_analy_rev_cit_list'] = resp_text


In [ ]:
import re
# extract all 'Yes', 'No'
pattern = r'"Pair\s*\d+"\s*:\s*"(Yes|No)"'

all_matches = []
for idx in range(len(df)):
    raw = df.at[idx, 'LLM_eval_master_agent_ext_analy_rev_cit_list']
    if not isinstance(raw, str):
        all_matches.append([])
        continue
    matches = re.findall(pattern, raw)
    all_matches.append(matches)


In [ ]:
import pandas as pd

rows = []
for idx, tuples in df['combined'].items():
    if not isinstance(tuples, list):
        continue
    # get the matching list for this row
    matches = all_matches[idx] if idx < len(all_matches) else []

    for j, (list1, list2) in enumerate(tuples):
        # grab the j-th match or None if out of range
        is_match = matches[j] if j < len(matches) else None

        rows.append({
            'source_row': idx,
            'List1':      list1,
            'List2':      list2,
            'is_match':   is_match
        })

result_df = pd.DataFrame(rows)

result_df.rename(
    columns={
        'List1': 'Ground_Truth',
        'List2': 'LLM_generated'
    },
    inplace=True
)


In [ ]:
import re

def extract_leading_number(x):
    """
    If x is a list, grab its first element; then:
    • If it’s an int, return it.
    • If it’s a string starting with digits (with or without a dot), return those digits.
    Otherwise return None.
    """
    # step 1: if it’s a list, pull out the first item
    val = x[0] if isinstance(x, list) and x else x

    # step 2: if it’s already an int, just return it
    if isinstance(val, int):
        return val

    # step 3: if it’s a string, regex for leading digits
    if isinstance(val, str):
        # match “123.” or just “123”
        m = re.match(r'^\s*(\d+)(?:\.)?', val)
        if m:
            return int(m.group(1))

    return None

# extract into new columns
result_df['gt_number']        = result_df['Ground_Truth'].apply(extract_leading_number)
result_df['llm_gen_number']   = result_df['LLM_generated'].apply(extract_leading_number)


In [ ]:
# ground truth coverage

# Initialize variables
current_section = None
section_has_yes = False
match = 0

# Iterate through the DataFrame
for index, row in result_df.iterrows():
    # Check if we are still in the same section
    if row['gt_number'] == current_section:
        # Check if there is a 'Yes' in 'is_match'
        if row['is_match'] == 'Yes':
            section_has_yes = True
    else:
        # We've reached a new section, check if the last section had a 'Yes'
        if section_has_yes:
            match += 1
        # Reset for new section
        current_section = row['gt_number']
        section_has_yes = (row['is_match'] == 'Yes')

# Check the last section after exiting the loop
if section_has_yes:
    match += 1
print(match)


# total number of unique ground truth

# Calculate consecutive blocks where 'ground_truth' is the same
unique_blocks = result_df['Ground_Truth'].ne(result_df['Ground_Truth'].shift()).cumsum()

# Group by these blocks and count each group
ck = result_df.groupby(unique_blocks)['gt_number'].agg(['count'])

# Output the results
print("Number of unique consecutive 'ground_truth' texts and their counts:")
print(ck)


Measuring Quality bewtween matched pairs (NLP based metrics)

Ground Truth: ground_truth || LLM_Generated limitation: llm_generated

In [ ]:
# say you want to rename 'oldA'→'newA' and 'oldB'→'newB'
df.rename(columns={
    'Ground_Truth': 'ground_truth',
    'LLM_generated': 'llm_generated',
    # 'Is_same': 'is_match',
}, inplace=True)

In [ ]:
# Drop rows where the column 'is_match' is 'no'
df_filtered = df[df['is_match'].str.lower() != 'no']

In [ ]:
df_filtered = df_filtered.reset_index(drop=True)

In [ ]:
# BERTScore (all)
!pip3 -q install bert-score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.5 MB/s eta 0:00:00


BERTScor for whole texts

In [ ]:
import pandas as pd
from bert_score import BERTScorer

# Initialize the BERT scorer
scorer = BERTScorer(model_type='roberta-large', lang="en")

# Function to calculate BERTScore for each row using one loop
def calculate_bertscore(row):
    # Calculate BERT Scores directly for the ground_truth and llm_generated of the row
    _, _, F1 = scorer.score([row['ground_truth']], [row['llm_generated']])
    return F1.mean().item()  # Return the mean F1 score

# Apply the function to each row in the DataFrame
df_filtered['bert_score'] = df_filtered.apply(calculate_bertscore, axis=1)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Calculate the average of the 'bert_score' column in df_highest_score
average_bert_score = df_filtered['bert_score'].mean()

# Display the average
average_bert_score


np.float64(0.8640587552331024)

In [ ]:
!pip3 -q install rouge_score

  Preparing metadata (setup.py) ... done


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import pandas as pd

# Initialize ROUGE scorer
rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Function to calculate similarity metrics for each row
def calculate_metrics(row):
    metrics = {}

    # ROUGE scores
    rouge_scores = rouge_scorer.score(row['ground_truth'], row['llm_generated'])
    metrics['rouge1'] = rouge_scores['rouge1'].fmeasure
    metrics['rouge2'] = rouge_scores['rouge2'].fmeasure
    metrics['rougeL'] = rouge_scores['rougeL'].fmeasure

    # Cosine Similarity
    vectorizer = CountVectorizer().fit_transform([row['ground_truth'], row['llm_generated']])
    vectors = vectorizer.toarray()
    metrics['cosine_similarity'] = cosine_similarity(vectors)[0, 1]

    # Jaccard Similarity
    set1 = set(row['ground_truth'].split())
    set2 = set(row['llm_generated'].split())
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    metrics['jaccard_similarity'] = intersection / union if union > 0 else 0

    # BLEU Score
    metrics['bleu_score'] = sentence_bleu([row['ground_truth'].split()], row['llm_generated'].split())

    return metrics

# Apply the function to each row in the DataFrame and store results in new columns
metric_results = df_filtered.apply(calculate_metrics, axis=1)

# Expand the dictionary into separate columns
metric_results_df = pd.DataFrame(metric_results.tolist())
df_filtered = pd.concat([df_filtered, metric_results_df], axis=1)


In [ ]:
# Calculate the average of each metric
average_metrics = {
    'Average ROUGE-1': df_filtered['rouge1'].mean(),
    'Average ROUGE-2': df_filtered['rouge2'].mean(),
    'Average ROUGE-L': df_filtered['rougeL'].mean(),
    'Average Cosine Similarity': df_filtered['cosine_similarity'].mean(),
    'Average Jaccard Similarity': df_filtered['jaccard_similarity'].mean(),
    'Average BLEU Score': df_filtered['bleu_score'].mean()
}

# Print the average metrics
average_metrics


In [ ]:
!pip3 -q install keybert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.2 MB/s eta 0:00:00


Topic similarity

In [ ]:
from keybert import KeyBERT

# Initialize KeyBERT model
kw_model = KeyBERT()

# Ensure all entries are strings (even if NaN)
df_filtered['ground_truth'] = df_filtered['ground_truth'].fillna("").astype(str)
df_filtered['llm_generated'] = df_filtered['llm_generated'].fillna("").astype(str)

# Now apply KeyBERT safely
# df_filtered['ground_truth_words'] = df_filtered['ground_truth'].apply(extract_keywords)
# df_filtered['LLM_generated_words'] = df_filtered['llm_generated'].apply(extract_keywords)


# Function to extract keywords using KeyBERT
def extract_keywords(text):
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words='english')
    return [kw[0] for kw in keywords]  # Extract just the keywords

# Apply KeyBERT to 'ground_truth' and 'LLM_generated' columns
df_filtered['ground_truth_words'] = df_filtered['ground_truth'].apply(extract_keywords)
df_filtered['LLM_generated_words'] = df_filtered['llm_generated'].apply(extract_keywords)


In [ ]:
# Function to compute Jaccard Similarity
def jaccard_similarity(row):
    set1 = set(row['ground_truth_words'])
    set2 = set(row['LLM_generated_words'])
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union > 0 else 0

# Apply Jaccard Similarity to each row
df_filtered['jaccard_similarity_topic'] = df_filtered.apply(jaccard_similarity, axis=1)
df_filtered['jaccard_similarity_topic'].mean()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Function to compute Cosine Similarity with empty check
def cosine_sim(row):
    gt = ' '.join(row['ground_truth_words'])
    llm = ' '.join(row['LLM_generated_words'])

    # If either is empty, return 0 similarity
    if not gt.strip() or not llm.strip():
        return 0.0

    try:
        vectorizer = CountVectorizer().fit_transform([gt, llm])
        vectors = vectorizer.toarray()
        return cosine_similarity(vectors)[0, 1]
    except ValueError:
        return 0.0  # fallback if vocabulary is still empty

df_filtered['cosine_similarity_topic'] = df_filtered.apply(cosine_sim, axis=1)
mean_sim = df_filtered['cosine_similarity_topic'].mean()
print(mean_sim)


In [ ]:
import re

# Function to extract text between double asterisks
def extract_text_between_asterisks(text):
    matches = re.findall(r'\*\*(.*?)\*\*', text)
    return matches

# Apply the function to both columns and store results in new columns
df_filtered['ground_truth_extracted'] = df_filtered['ground_truth'].apply(extract_text_between_asterisks)
df_filtered['llm_generated_extracted'] = df_filtered['llm_generated'].apply(extract_text_between_asterisks)


### Evaluation: Faithfulness, Soundness, and Importance

In [ ]:
evaluation_prompt_template = '''
You are an expert reviewer. Evaluate the quality of the generated limitations based on the following three criteria: Faithfulness,
Soundness, and Importance. For each criterion, assign a score between 1 and 5 and provide a short justification.

Faithfulness = The generated limitations should accurately represent the paper’s content and findings, avoiding any introduction
of misinformation or contradictions to the original concepts, methodologies or results presented.
– 5 points: Perfect alignment with the original content and findings, with no misinformation or contradictions. Fully reflects the
paper’s concepts, methodologies, and results
accurately.
– 4 points: Mostly aligns with the original content but contains minor inaccuracies or slight
misinterpretations. These do not significantly
affect the overall understanding of the paper’s
concepts or results.
– 3 points: Generally aligns with the original
content but includes several minor inaccuracies or contradictions. Some elements may
not fully reflect the paper’s concepts or results,
though the overall understanding is mostly intact.
– 2 points: Noticeable misalignment with the
original content, with multiple inaccuracies
or contradictions that could mislead readers.
Some key aspects of the paper’s concepts or
results are misrepresented.
– 1 point: Introduces significant misalignment
by misrepresenting issues that do not exist in
the paper. Creates considerable misinformation and contradictions that distort the original
content, concepts, or results.

Soundness = The generated limitations should be detailed and specific, with suggestions or critiques that are practical, logically
coherent, and purposeful. It should clearly address relevant aspects of the paper and offer insights that can genuinely improve the
research.
– 5 points: Highly detailed and specific, with
practical, logically coherent, and purposeful
suggestions. Clearly addresses relevant aspects and offers insights that substantially improve the research.
– 4 points: Detailed and mostly specific, with
generally practical and logically sound suggestions. Addresses relevant aspects well but may
lack depth or novelty in some areas.
– 3 points: Detailed and specific but with some
issues in practicality or logical coherence. Suggestions are somewhat relevant and offer partial improvements.
– 2 points: Somewhat vague or lacking in specificity, with suggestions that have limited practicality or logical coherence. Addresses
relevant aspects only partially and provides minimal improvement.
– 1 point: Lacks detail and specificity, with impractical or incoherent suggestions. Fails to
effectively address relevant aspects or offer
constructive insights for improvement.

Importance =  The generated limitations should
address the most significant issues that impact the
paper’s main findings and contributions. They
should highlight key areas where improvements
or further research are needed, emphasizing their
potential to enhance the research’s relevance and
overall impact.
– 5 points: Addresses critical issues that substantially impact the paper’s findings and contributions. Clearly identifies major areas for
significant improvement or further research,
enhancing the research’s relevance and overall
impact.
– 4 points: Identifies meaningful issues that contribute to refining the paper’s findings and
methodology. While the impact is notable,
it does not reach the level of fundamentally
shaping future research directions.
– 3 points: Highlights important issues that offer some improvement to the current work but
do not significantly impact future research directions. Provides useful insights for refining
the paper but lacks broader implications for
further study.
– 2 points: Points out limitations with limited
relevance to the paper’s overall findings and
contributions. Suggestions offer marginal improvements but fail to address more substantial
gaps in the research.
– 1 point: Focuses on trivial issues, such as minor errors or overly detailed aspects. Does not
address substantive issues affecting the paper’s
findings or contributions, limiting its overall
relevance and impact.

Input:
Input Paper: [Input Paper]
LLM Generated Limitations: [LLM Generated Limitations]

Please evaluate the **Generated Limitations** based on the **Input Paper Content** and return your response strictly in the following JSON format:

Faithfulness: rating: , explanation:,
Soundness:    rating: explanation: ,
Importance:   rating: , explanation:

'''


In [ ]:
import os
import base64
import time
import pandas as pd
from openai import AzureOpenAI, RateLimitError
from openai import OpenAI

# Set up OpenAI API
os.environ['OPENAI_API_KEY'] = ''
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# Define the OpenAI streaming function for GPT-4o-mini
def run_critic_openai(prompt: str):
    summary_text = ""
    stream = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt}
        ],
        stream=True,
        temperature=0
    )
    for chunk in stream:
        summary_text += chunk.choices[0].delta.content or ""
    return summary_text.strip()


Measuring Faithfulness, Sundness, and Importance between Input text (response_string_neurips) and LLM Generated limitations (master_agent_ext_analy_rev_cit_with_rel)

In [ ]:
# results = []

df['ext_analy_rev_cit_with_rel_result'] = ''
for i in range(len(df)): # len(df)
    print("i is",i)
    input_text = df.at[i, 'response_string_neurips']
    generated_limitations = df.at[i, 'master_agent_ext_analy_rev_cit_with_rel']

    if pd.isna(input_text) or pd.isna(generated_limitations):
        results.append(None)
        continue

    prompt = evaluation_prompt_template.format(
        input_text=input_text.strip(),
        generated_limitations=generated_limitations.strip()
    )

    try:
        result = azure_run_critic(prompt)
    except Exception as e:
        print(f"Error at row {i}: {e}")
        result = None

    df.at[i, "ext_analy_rev_cit_with_rel_result"] = result


Extracting the Faithfulness, Soundness, and Importance score

In [ ]:
import pandas as pd
import json
import re

# Initialize empty columns
df['faithfulness_score'] = None
df['soundness_score'] = None
df['importance_score'] = None

for idx, row in df.iterrows():
    val = row['ext_analy_rev_cit_with_rel_result']

    if pd.isna(val):
        continue

    try:
        # Clean and parse JSON
        clean_json = re.sub(r'```json|```', '', val).strip()
        parsed = json.loads(clean_json)

        # Store ratings into new columns
        df.at[idx, 'faithfulness_score'] = parsed['Faithfulness']['rating']
        df.at[idx, 'soundness_score'] = parsed['Soundness']['rating']
        df.at[idx, 'importance_score'] = parsed['Importance']['rating']

    except Exception as e:
        print(f"Row {idx} failed to parse:", e)
        continue

avg_faith = df['faithfulness_score'].mean()
avg_sound = df['soundness_score'].mean()
avg_imp = df['importance_score'].mean()

print(f"Average Faithfulness: {avg_faith:.2f}")
print(f"Average Soundness:   {avg_sound:.2f}")
print(f"Average Importance:  {avg_imp:.2f}")


Measuring score with Ground truth (Lim_and_OR_ground_truth_final) and LLM Generated response (master_agent_ext_analy_rev_cit_with_rel) by providing score from previous response (input and LLM generated text)

In [ ]:
import json
import re

# Initialize result columns
df['adjusted_score_ext_analy_rev_cit_with_rel_json'] = None  # Optional: for storing raw JSON string

for i in range(len(df)): # len(df)
    print("i is",i)
    try:
        input_text = df.at[i, 'Lim_and_OR_ground_truth_final']
        generated = df.at[i, 'master_agent_ext_analy_rev_cit_with_rel']
        faith = df.at[i, 'faithfulness_score']
        sound = df.at[i, 'soundness_score']
        imp = df.at[i, 'importance_score']

        if pd.isna(input_text) or pd.isna(generated) or pd.isna(faith) or pd.isna(sound) or pd.isna(imp):
            continue

        prompt = evaluation_prompt_template.format(
            input_text=input_text.strip(),
            generated_limitations=generated.strip(),
            faith=int(faith),
            sound=int(sound),
            imp=int(imp)
        )

        result = azure_run_critic(prompt)
        df.at[i, 'adjusted_score_ext_analy_rev_cit_with_rel_json'] = result  # Optional: Store full JSON output

    except Exception as e:
        print(f"Row {i} failed: {e}")
        continue



In [ ]:
import re

# Initialize new columns
df['adjusted_faithfulness_score'] = None
df['adjusted_soundness_score'] = None
df['adjusted_importance_score'] = None

# Define regex patterns for each score
faith_re = re.compile(r'"Faithfulness"\s*:\s*{\s*"rating"\s*:\s*(\d+)', re.DOTALL)
sound_re = re.compile(r'"Soundness"\s*:\s*{\s*"rating"\s*:\s*(\d+)', re.DOTALL)
imp_re   = re.compile(r'"Importance"\s*:\s*{\s*"rating"\s*:\s*(\d+)', re.DOTALL)

# Apply regex extraction row-wise
for i in range(len(df)):
    row = df.at[i, 'adjusted_score_ext_analy_rev_cit_with_rel_json']
    if pd.isna(row):
        continue

    # Clean text from triple backticks and newline artifacts
    cleaned = re.sub(r"```json|```", "", row).strip()

    # Extract values using regex
    faith_match = faith_re.search(cleaned)
    sound_match = sound_re.search(cleaned)
    imp_match   = imp_re.search(cleaned)

    if faith_match:
        df.at[i, 'adjusted_faithfulness_score'] = int(faith_match.group(1))
    if sound_match:
        df.at[i, 'adjusted_soundness_score'] = int(sound_match.group(1))
    if imp_match:
        df.at[i, 'adjusted_importance_score'] = int(imp_match.group(1))


In [ ]:
avg_faith = df['adjusted_faithfulness_score'].mean()
avg_sound = df['adjusted_soundness_score'].mean()
avg_imp = df['adjusted_importance_score'].mean()

print(f"Average Faithfulness: {avg_faith:.2f}")
print(f"Average Soundness:   {avg_sound:.2f}")
print(f"Average Importance:  {avg_imp:.2f}")
